In [18]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from IPython.display import display, HTML
import os
import re

notebook_path = os.path.abspath("adhoc.ipynb")
notebook_dir = str(os.path.dirname(notebook_path)).replace("\\\\", "/").replace("\\", "/")
output_folder = f"{notebook_dir}/../output"
raw_output = f"{output_folder}/raw"
curated_output = f"{output_folder}/curated"
semantic_output = f"{output_folder}/semantic"

nyse_tickers = f"{notebook_dir}/../wsb/nyse-listed.csv"
nasdaq_tickers = f"{notebook_dir}/../wsb/nasdaq-listed.csv"
curated_stockticker = f"{curated_output}/StockTicker.csv"

# output_html = f"{notebook_dir}/../index.html"

In [19]:
pd.set_option('display.max_colwidth', None)
sep = "|"

def convert_list(x):
    return x.strip("[]").split(", ")

df = pd.read_csv(curated_stockticker, sep=sep, 
converters={"title_ticker": lambda x: convert_list(x), 
"submission_text_ticker": lambda x: convert_list(x)}
)

df.dtypes
# display(df)
# display(df.iloc[[0]])

Unnamed: 0                  int64
id                         object
title                      object
name                       object
upvote_ratio              float64
ups                         int64
score                       int64
sort                       object
created                    object
author                     object
num_comments                int64
flair                      object
permalink                  object
built_url                  object
url                        object
submission_text            object
last_updated               object
raw_filename               object
model                      object
title_regex                object
title_ticker               object
submission_text_regex      object
submission_text_ticker     object
dtype: object

In [20]:
df[df['id'].str.contains("l5gsp8")][["title","built_url","title_ticker", "submission_text_ticker"]]


,title,built_url,title_ticker,submission_text_ticker


In [21]:

def explode(df, cols):
        """method to explode list columns. mainly used for stock ticker columns

        :param df: pandas df
        :type df: obj
        :param cols: list of cols to explode
        :type cols: list
        """
        for col in cols:
            df = df.explode(col)

        return df

ticker_cols = ["title_ticker", "submission_text_ticker"]

exploded_df = explode(df, ["title_ticker", "submission_text_ticker"])
# display(exploded_df)

In [55]:
# https://stackoverflow.com/questions/23415500/pandas-plotting-a-stacked-bar-chart

# pd.pivot_table(exploded_df, values='id', index=ticker_cols, columns=['C'], aggfunc=np.sum)

title_df = exploded_df[["id", "title", "built_url", "title_ticker"]].drop_duplicates(subset=['id', 'title_ticker'])
title_df["title/submission"] = "title"
title_df = title_df.rename({"title_ticker": "ticker"},axis=1)

# title_df.groupby("title_ticker").nunique()
# agg_title = title_df[["id", "ticker", "title/submission"]].groupby(["ticker", "title/submission"]).agg(['count']).reset_index()

submission_df = exploded_df[["id", "title", "built_url", "submission_text_ticker"]].drop_duplicates(subset=['id','submission_text_ticker'])
submission_df["title/submission"] = "submission"
submission_df = submission_df.rename({"submission_text_ticker": "ticker"}, axis=1)

# agg_submission = submission_df[["id", "ticker", "title/submission"]].groupby(["ticker", "title/submission"]).agg(['count']).reset_index()


# agg_title.merge(agg_submission, on="ticker", how="outer")
# pd.concat([agg_submission,agg_title], ignore_index=True)
plot_df = pd.concat([title_df, submission_df], ignore_index=True)


# concat_df = concat_df[["ticker", "title/submission"]]

def filter_count(df, col, min):
    return df[df.groupby(col)[col].transform('count') > min]

plot_df = filter_count(plot_df, "ticker", 1)
# plot_df = plot_df.groupby(["ticker", "title/submission"])["ticker"].count().unstack('title/submission').fillna(0).reset_index()
# .plot(kind='bar', stacked=True).figure.savefig(f'{notebook_dir}/StockTicker.png')

plot_df = plot_df.drop_duplicates(subset=['id'])
plot_df = plot_df[plot_df['ticker'].str.strip().astype(bool)]
plot_df

,id,title,built_url,ticker,title/submission
2,l78uct,GME YOLO update — Jan 28 2021,https://www.reddit.com/r/wallstreetbets/comments/l78uct/gme_yolo_update_jan_28_2021/,'GME',title
4,l76oiz,YOU RETARDS ARE ACTUALLY HOLDING THE LINE!!!! DON'T STOP 💎💎💎💎 HANDS!!!!,https://www.reddit.com/r/wallstreetbets/comments/l76oiz/you_retards_are_actually_holding_the_line_dont/,'ARE',title
21,l75z6q,"This guy is the SOLE OWNER of Point72, a VC operating w $3BILLION of Citadel $ and invested $750 Million shorting GME... oh ya and he owns the PoS Met’s. Can we ruin this Toxic Boomer? PLEASE💩🛸 🚀🚀🚀",https://www.reddit.com/r/wallstreetbets/comments/l75z6q/this_guy_is_the_sole_owner_of_point72_a_vc/,'GME',title
26,l73tdc,ROBINHOOD HAS BEEN SUED! OTHERS TO FOLLOW!!!,https://www.reddit.com/r/wallstreetbets/comments/l73tdc/robinhood_has_been_sued_others_to_follow/,'HAS',title
31,l76e2g,WeBull Opened for Trading GME and AMC!,https://www.reddit.com/r/wallstreetbets/comments/l76e2g/webull_opened_for_trading_gme_and_amc/,'GME',title
...,...,...,...,...,...
3726,l7897o,"Did Point69 threaten the entire market if their losses weren't capped? ""ReTaIL VoLaTIliTy"" 😂 What a joke. Watch out for the 3:45pm sell spike if Cohen's still Angry. #id giveGMEstockforsometrumptweetsrn",https://www.reddit.com/r/wallstreetbets/comments/l7897o/did_point69_threaten_the_entire_market_if_their/,'B',submission
4404,l79i3h,"NVAX just announced their Interim results! Incredible news, we have a 3rd vaccine that is on par with Pfizer and Moderna in terms of efficacy.",https://www.reddit.com/r/wallstreetbets/comments/l79i3h/nvax_just_announced_their_interim_results/,'DD',submission
5102,l71kws,Can WSB bring AMZN to $0?,https://www.reddit.com/r/wallstreetbets/comments/l71kws/can_wsb_bring_amzn_to_0/,'GME',submission
5487,l6xd4k,"Sabre Corporation can get squoze, $SABR, still 50% down despite monopoly over travel",https://www.reddit.com/r/wallstreetbets/comments/l6xd4k/sabre_corporation_can_get_squoze_sabr_still_50/,'AAL',submission


In [64]:
import altair as alt

# use with caution: https://altair-viz.github.io/user_guide/faq.html#maxrowserror-how-can-i-plot-large-datasets
# alt.data_transformers.disable_max_rows()
alt.renderers.set_embed_options(theme='dark')

# brush = alt.selection(type='interval')
selector = alt.selection_single(empty='all', fields=['ticker'])
base = alt.Chart(plot_df).add_selection(selector)

bars = base.mark_bar().encode(
    x='ticker',
    y="count(id)"
).properties(
    width=1700,
    height=600
)

# base chart for data tables
ranked_text = base.mark_text().encode(
    y=alt.Y('row_number:O',axis=None)
).transform_window(
    row_number='row_number()'
).transform_filter(
    selector
).transform_window(
    rank='rank(row_number)'
).transform_filter(
    # only shows up to 20
    alt.datum.rank<20
)


# Data Tables
ticker = ranked_text.encode(text='ticker').properties(title='Stock Ticker')
title = ranked_text.encode(text='title').properties(title='Submission Title')
url = ranked_text.encode(text='built_url').properties(title='URL')

# Combine data tables
text = alt.hconcat(ticker, title, url)

# Build chart
chart = alt.vconcat(
    bars,
    text,
    # autosize="fit"
).resolve_legend(
    color="independent"
)

chart
# chart.to_json(indent=None)

alt.VConcatChart(...)

In [23]:
# drill_df = plot_df.groupby(["ticker", "title/submission"])["ticker"].count().unstack('title/submission').fillna(0).reset_index()

# for row in drill_df.itertuples():
#     temp_data = [
#         ["submission_text", row.submission],
#         ["title", row.title]
#         ]
#     # print(f"{row.ticker}: {row.submission}")
#     # print(f"{row.ticker}: {row.title}")
#     H.add_drilldown_data_set(temp_data, 'column', row.ticker, name=row.ticker )

In [24]:
agg_df = plot_df[["ticker", "id"]].groupby(["ticker"]).nunique().reset_index()

# display(agg_df)
agg_df["drilldown"] = agg_df["ticker"]
data = agg_df.rename({"id": "y", "ticker": "name"}, axis=1).to_dict('records')

In [25]:
stack_df = plot_df.groupby(["ticker", "title/submission"])["ticker"].count().unstack('title/submission').fillna(0).reset_index()
stack_df

title/submission,ticker,submission,title
0,,1581.0,1372.0
1,'AA',1.0,1.0
2,'AAL',7.0,5.0
3,'AAPL',2.0,0.0
4,'ACH',2.0,0.0
...,...,...,...
134,'WB',1.0,1.0
135,'WIN',6.0,4.0
136,'X',8.0,1.0
137,'XL',1.0,1.0


In [33]:
import altair as alt

# load a simple dataset as a pandas DataFrame
from vega_datasets import data
cars = data.cars()
cars[:1]

,Name,Miles_per_Gallon,Cylinders,Displacement,Horsepower,Weight_in_lbs,Acceleration,Year,Origin
0,chevrolet chevelle malibu,18.0,8,307.0,130.0,3504,12.0,1970-01-01,USA


In [26]:
import altair as alt

# load a simple dataset as a pandas DataFrame
from vega_datasets import data
cars = data.cars()

alt.Chart(cars).mark_point().encode(
    x='Horsepower',
    y='Miles_per_Gallon',
    color='Origin',
).interactive()

alt.Chart(...)

In [27]:
import pandas as pd
wide_form = pd.DataFrame({'Date': ['2007-10-01', '2007-11-01', '2007-12-01'],
                          'AAPL': [189.95, 182.22, 198.08],
                          'AMZN': [89.15, 90.56, 92.64],
                          'GOOG': [707.00, 693.00, 691.48]})
print(wide_form)

         Date    AAPL   AMZN    GOOG
0  2007-10-01  189.95  89.15  707.00
1  2007-11-01  182.22  90.56  693.00
2  2007-12-01  198.08  92.64  691.48


In [28]:
long_form = pd.DataFrame({'Date': ['2007-10-01', '2007-11-01', '2007-12-01',
                                   '2007-10-01', '2007-11-01', '2007-12-01',
                                   '2007-10-01', '2007-11-01', '2007-12-01'],
                          'company': ['AAPL', 'AAPL', 'AAPL',
                                      'AMZN', 'AMZN', 'AMZN',
                                      'GOOG', 'GOOG', 'GOOG'],
                          'price': [189.95, 182.22, 198.08,
                                     89.15,  90.56,  92.64,
                                    707.00, 693.00, 691.48]})
print(long_form)

         Date company   price
0  2007-10-01    AAPL  189.95
1  2007-11-01    AAPL  182.22
2  2007-12-01    AAPL  198.08
3  2007-10-01    AMZN   89.15
4  2007-11-01    AMZN   90.56
5  2007-12-01    AMZN   92.64
6  2007-10-01    GOOG  707.00
7  2007-11-01    GOOG  693.00
8  2007-12-01    GOOG  691.48


In [29]:
alt.Chart(long_form).mark_line().encode(
  x='Date:T',
  y='price:Q',
  color='company:N'
)

alt.Chart(...)